# GAPM for LandS problem
This notebook shows an implementation of the Generalized Adaptive Partition Method for the LandS problem. 

Disclaimer: I tried to keep it simple, using standard data structures and straigthforward formulations. Many steps can be done more efficiently. 

This notebook requires Gurobi (https://www.gurobi.com) as optimization solver.

In [1]:
import gurobipy as gp
import math
from gurobipy import GRB
from gurobipy import quicksum

We start creating the scenarios and the other data of the problem

In [2]:
nscen = 1000000
scen_full = []
prob_full = []
for i in range (1,101):
    for j in range(1,101):
        for k in range(1,101):
            scen_full.append([0.04*(i-1),0.04*(j-1),0.04*(k-1)])
            prob_full.append(1/nscen)
cost = [10,7,16,6]
flow = [[40,24,4],
        [45,27,4.5],
        [32,19.2,3.2],
        [55,33,5.5]]
budget = 120
minCap = 12
nx = 4
ny = 3

We create a *Master* problem, which is the deterministic equivalent formulation of the LandS problem, but for an arbitrary set of scenarios $S$, each of them with its corresponding probability $p_s$. 

The formulation of this problem is the following:

$$[MP] := \min_{x,y \geq 0}   \sum_{i=1}^4  cost_i x_i + \sum_{s\in S} p_s \sum_{i=1}^3 \sum_{j=1}^3flow_{ij}y_{ij}^s$$
$$\sum_{i=1}^4  x_i \geq minCap$$
$$\sum_{i=1}^4 c_ix_i \leq budget$$
$$\sum_{j=1}^3 y_{ij}^s \leq x_i ~\forall~ i\in 1\ldots 4, \forall s\in S$$
$$\sum_{i=1}^4 y_{ij}^s \geq d_j^s  ~\forall~ j\in 1\ldots 3,  \forall s\in S$$

The master problem solve this formulation, and return the solution and the objective.

In [3]:
def Master(scen,prob):
    nz = len(scen)
    m = gp.Model("master")
    m.Params.OutputFlag = 0
    Xvar = m.addVars(nx,vtype=GRB.CONTINUOUS,name='x')
    Yvar = m.addVars(nx,ny,nz,vtype=GRB.CONTINUOUS, name = 'y')
    consBudget = m.addConstr(quicksum(Xvar[i]*cost[i] for i in range(nx)) <= budget)
    consMinCap = m.addConstr(quicksum(Xvar[i] for i in range(nx)) >= minCap)
    consCap = m.addConstrs(quicksum(Yvar[i,j,k] for j in range(ny)) <= Xvar[i] for i in range(nx) for k in range(nz))
    for j in range(0,ny):
        consFlow = m.addConstrs(quicksum(Yvar[i,j,k] for i in range(nx)) >= scen[k][j] for k in range(nz))
    m.setObjective(quicksum(Xvar[i]*cost[i] for i in range(nx)) + quicksum(quicksum(prob[k]*Yvar[i,j,k]*flow[i][j]
    for j in range(ny) for k in range(nz)) for i in range(nx)), GRB.MINIMIZE)
    m.optimize()
    LB = m.objVal
    x = [Xvar[i].x for i in range(nx)]
    return(x,LB)

The subproblem is the second-stage problem for a fixed $\hat{x}$. That is
$$[SP] := \sum_{i=1}^3 \sum_{j=1}^3flow_{ij}y_{ij}^s$$
$$\sum_{j=1}^3 y_{ij}^s \leq \hat{x}_i ~\forall~ i\in 1\ldots 4, \forall s\in S$$
$$\sum_{i=1}^4 y_{ij}^s \geq d_j^s  ~\forall~ j\in 1\ldots 3,  \forall s\in S$$

We solve this problem for each scenarios $s\in S$ and return the objective value and the duals. In this case, we only requires the dual of the second constraints (see [Ramirez-Pico & Moreno, 2022](https://link.springer.com/article/10.1007/s10107-020-01609-8). The sum of the objective of each subproblem, plus the cost of the current first-stage solution $\hat{x}$  provides an upper bound for the problem.

In [14]:
def Sub(x):
    nz = len(scen_full)
    m = gp.Model("subproblem")
    m.Params.OutputFlag = 0
    Yvar = m.addVars(nx,ny,vtype=GRB.CONTINUOUS, name = 'y')
    consCap = m.addConstrs(quicksum(Yvar[i,j] for j in range(ny)) <= x[i] for i in range(nx))
    consFlow = m.addConstrs(quicksum(Yvar[i,j] for i in range(nx)) >= 0 for j in range(ny))
    m.setObjective(quicksum(Yvar[i,j]*flow[i][j] for i in range(nx) for j in range(ny))) 
    duals = []
    UB = sum([x[i]*cost[i] for i in range(nx)])
    for k in range(nscen):
        #if (k%100000)==0:
        #    print("Solving subproblem %d" % k)
        for j in range(ny):
            consFlow[j].RHS = scen_full[k][j]
        m.optimize()
        UB += prob_full[k]*m.objVal
        dualFlow = tuple([consFlow[j].pi for j in range(ny)])
        duals.append(dualFlow)
    return(duals,UB)

Finally, starting with a partition of the scenarios, for each subset of the partition we dissagregate its scenarios into subsets sharing the same dual variables.

In [6]:
def refine(duals, partition_prev):
    partition_new = []
    for i in list(set(duals)):
        partition_new.append([k for k in range(len(scen_full)) if duals[k] == i])
    partition = []
    for i in partition_prev:
        for j in partition_new:
            sub = set(i).intersection(set(j))
            if sub != set():
                partition.append(list(sub))
    # We recompute the average of each subset and the probability of them
    Aggscen = []
    prob = []
    for subset in partition:
        probLocal = 0
        scen = [0,0,0]
        for k in subset:
            for j in range(ny):
                scen[j] += prob_full[k]*scen_full[k][j]
            probLocal += prob_full[k]
        for j in range(ny):
            scen[j] = scen[j]/probLocal       
        Aggscen.append(scen)
        prob.append(probLocal)
    return(Aggscen,prob,partition)
    

We iterate to apply the GAPM. We start with a partition containing all scenarios, and iterate solving the master (to get an $x$ and a lower bound), solving the subproblems (to get the duals and an upper bound) and refining the partition. You can define a stoping criteria when LB ~ UB.

In [9]:
def Gap(UB,LB):
    return(abs(UB-LB)*100/UB)

In [12]:
def iter(max_iter = 100, stop_gap = 1e-6):
    aux = 0
    UB = 1e6
    LB = 0
    aux = 0
    # initial partition containig all scenarios
    Part_aux = [list(range(len(scen_full)))]
    Agg_aux = [[sum([scen_full[k][j]*prob_full[k] for k in range(len(scen_full))]) for j in range(ny)]]
    prob_aux = [sum(prob_full)]
    Gaps = []
    UBs = []
    LBs = []
    Aggs = []
    Sols = []
    Parts = []
    while(aux <= max_iter and Gap(UB,LB) > stop_gap):
        aux = aux + 1
        (Master_aux, LB) = Master(Agg_aux,prob_aux)
        (duals, UB) = Sub(Master_aux)
        (Agg_aux, prob_aux,Part_aux) = refine(duals,Part_aux)
        print("Iter:%3d LB=%3.4f UB=%3.4f GAP=%1.5f%% Part_size=%3d" % (aux,LB,UB,Gap(UB,LB),len(Part_aux)))
        UBs.append(UB)
        LBs.append(LB)
        Gaps.append(Gap(UB,LB))
        Aggs.append(Agg_aux)
        Sols.append(Master_aux)
        Parts.append(Part_aux)
    return (UBs,LBs,Gaps,Aggs,Sols,Parts)

And we iterate. Warning: solving the subproblems can take a long time depending on your computer.

In [15]:
(UBs,LBs,Gaps,Aggs, Sols, Parts) = iter(20)

Iter:  1 LB=221.4900 UB=228.6969 GAP=3.15129% Part_size= 17
Iter:  2 LB=224.5183 UB=225.9386 GAP=0.62864% Part_size= 72
Iter:  3 LB=225.3638 UB=225.7268 GAP=0.16083% Part_size=179
Iter:  4 LB=225.5180 UB=225.6354 GAP=0.05203% Part_size=369
Iter:  5 LB=225.6155 UB=225.6362 GAP=0.00919% Part_size=657
Iter:  6 LB=225.6259 UB=225.6307 GAP=0.00215% Part_size=1048
Iter:  7 LB=225.6285 UB=225.6299 GAP=0.00060% Part_size=1373
Iter:  8 LB=225.6294 UB=225.6294 GAP=0.00000% Part_size=1374


We can see the evolution of the solution found at each iteration

In [16]:
for i in Sols:
    print("%1.4f %1.4f %1.4f %1.4f" % (i[0], i[1], i[2],i[3]))

1.9800 0.0000 1.9800 8.0400
1.6072 2.4334 1.4848 6.4745
0.7994 4.0063 1.7406 5.4537
0.9600 3.2439 1.8600 5.9361
0.7600 3.5467 1.9200 5.7733
0.8600 3.3800 1.9000 5.8600
0.8600 3.3601 1.8800 5.8999
0.8400 3.4000 1.8800 5.8800


And the objetive value

In [17]:
for i in list(zip(UBs,LBs,Gaps)):
    print("%3.4f %3.4f %1.5f%%" % (i[0], i[1], i[2]))

228.6969 221.4900 3.15129%
225.9386 224.5183 0.62864%
225.7268 225.3638 0.16083%
225.6354 225.5180 0.05203%
225.6362 225.6155 0.00919%
225.6307 225.6259 0.00215%
225.6299 225.6285 0.00060%
225.6294 225.6294 0.00000%


We store the final partition, so we can validate the solution found using an exact solver.

In [27]:
partFinal = Parts[-1:][0]
AggFinal = Aggs[-1:][0]
probFinal = [len(partFinal[k])/nscen for k in range(len(partFinal))]

In [30]:
with open('partition.txt', 'w') as f: 
    for k in range(len(partFinal)):
        f.write("%d: %0.6f" % (k,probFinal[k])) # id and its probability
        for j in range(ny):
            f.write(" %0.6f" % AggFinal[k][j]) # aggregated demand
        for s in partFinal[k]:
            f.write(" %d" % s) #scenarios in the partition
        f.write("\n")
